In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_excel('cleaned_CB.xlsx')
df

,age,avg_order_value,total_orders,is_fraudulent,loyalty_score,churn_risk,customer_since_month,last_purchase_month,gender_Male,gender_Other,...,country_France,country_Germany,country_India,country_Japan,country_UK,country_USA,preferred_category_Electronics,preferred_category_Fashion,preferred_category_Home,preferred_category_Sports
0,-1.015895,0.150485,8,1,-0.001367,-0.529103,-0.997086,-0.150141,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.270530,-0.029012,10,0,-0.452297,0.347683,-0.770553,-1.009074,0,0,...,0,0,0,0,0,1,1,0,0,0
2,1.389160,-0.158530,6,0,0.518938,-1.468516,-0.883820,0.136169,1,0,...,0,0,0,0,0,0,0,0,0,1
3,-1.015895,0.286960,9,1,1.490173,-0.591730,1.721313,1.567724,0,1,...,0,0,0,1,0,0,1,0,0,0
4,-1.071827,1.871204,16,0,1.004556,-0.842241,-1.563419,1.567724,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.046803,0.594432,9,0,-1.284785,0.285056,1.551413,1.281413,0,0,...,0,0,0,0,1,0,1,0,0,0
4996,-1.407416,-0.992253,10,0,0.553625,-0.403848,-1.223619,0.995102,0,0,...,0,0,0,0,1,0,0,0,1,0
4997,1.724749,1.534210,16,0,0.206755,-0.341220,0.078947,-1.009074,1,0,...,0,0,0,1,0,0,0,0,0,0
4998,0.773913,1.495854,8,0,-0.105428,-1.218006,-0.260853,0.708791,1,0,...,0,0,0,0,0,1,0,1,0,0


In [3]:
X = df.drop(columns = ['is_fraudulent'])
y = df['is_fraudulent']

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,random_state = 81)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
estimator = GradientBoostingClassifier(random_state = 81)
param_grid = {"n_estimators":list(range(1,10)),
             'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}

gb_grid = GridSearchCV(estimator,param_grid,cv=5,scoring='accuracy')
gb_grid.fit(X_train,y_train)
gb_grid.best_params_

{'learning_rate': 0.1, 'n_estimators': 1}

In [7]:
gb_grid.best_estimator_.feature_importances_

array([0.15134529, 0.        , 0.        , 0.        , 0.0591019 ,
       0.0673803 , 0.        , 0.        , 0.40201008, 0.        ,
       0.        , 0.        , 0.        , 0.14154595, 0.        ,
       0.17861647, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [8]:
feature = pd.DataFrame(data = gb_grid.best_estimator_.feature_importances_,
                       index = X_train.columns,
                       columns = ["Feature_importances"])
feature_imp = feature[feature["Feature_importances"] > 0]
imp_features = feature_imp.index.tolist()
imp_features

['age',
 'churn_risk',
 'customer_since_month',
 'gender_Other',
 'country_Germany',
 'country_Japan']

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
X2 = X[imp_features]

X2_train,X2_test,y2_train,y2_test = train_test_split(X2,y,train_size=0.8,random_state=81)

gb_model = GradientBoostingClassifier(n_estimators = 1,learning_rate = 0.1)
gb_model.fit(X2_train,y2_train)

ypred_train = gb_model.predict(X2_train)
ypred_test = gb_model.predict(X2_test)

print('train_accuracy',accuracy_score(ypred_train,y2_train))
print('test_accuracy',accuracy_score(ypred_test,y2_test))
print('cross_validation_score',cross_val_score(gb_model,X2_train,y2_train,scoring='accuracy',cv=5).mean())

train_accuracy 0.9755
test_accuracy 0.971
cross_validation_score 0.9737499999999999
